In [1]:
import os
%load_ext autoreload
%autoreload 2
import sys
import time

# Adjacent NN for motion segmentation
from data.dataloader import SFDataset4D, compensate_ego_motion, construct_global_poses_from_relative
from pytorch3d.ops.knn import knn_points
from vis.deprecated_vis import *
from loss.flow import DT

import torch
import numpy as np
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from torch_scatter import scatter
from ops.metric import SceneFlowMetric
from models.RSNF import RigidNeuralPriorV2, NeuralPrior
from models.jointST import STNeuralPrior, MultiRigidNeuralPrior
from benchmark.evaluation import get_datetime

plt.close()

# Data
device = torch.device('cuda:0')
motion_metric = 0.05
eps = 0.15

max_flow_epoch = 200
loss_diff_stop = 0.001
early_patience = 100
verbose = True
COMPENSATE_EGO = False

dataset = SFDataset4D(dataset_type='waymo', n_frames=1, only_first=True)


exp_folder = f'/mnt/personal/vacekpa2/experiments/' + 'development_metric'
os.makedirs(f'{exp_folder}', exist_ok=True)
runs = 1

for run in range(runs):
    cfg = {'param' : 1, 'eps' : 0.15, 'run' : run}
    My_metric = SceneFlowMetric()
        
    for i, data in enumerate(tqdm(dataset)):
        data = dataset[4]
        
        if COMPENSATE_EGO: # with options
            data = compensate_ego_motion(data)
            # missing compensation to pc2
            data['pc1'] = data['global_pc1']
            data['pc2'] = data['global_pc2']
             
        # Unpack Data
        data['pc1'] = data['pc1'].to(device)
        data['pc2'] = data['pc2'].to(device)
        data['gt_flow'] = data['gt_flow'].to(device)
        data['id_mask1'] = data['id_mask1'].to(device)
        
        
        
        # model = RigidNeuralPriorV2(lr=0.008, early_stop=100, verbose=True).to(device) # better!
        # model = NeuralPrior(lr=0.008, early_stop=500, verbose=False).to(device)
        # model = STNeuralPrior(eps=eps, lr=0.003, cluster_every_iter=1000, refine=False, early_stop=500, verbose=False).to(device)
        model = MultiRigidNeuralPrior(eps=eps, lr=0.008, cluster_every_iter=10000, refine=False, early_stop=10, verbose=False).to(device)
        
    
        # visualize_multiple_pcls(*[a.detach().cpu().numpy(), pc2[0].detach().cpu().numpy()])
        
        output_data = model(data)
        My_metric.update(output_data)
        # print('\r\033[2K\033[1G', end='', flush=True)   # this shift to one line up
    
        if i == 0: break
        
    # store metrics as file and plot?
    # visualize_flow3d(data['pc1'][0], data['pc2'][0], data['pred_flow'][0])
    # visualize_points3D(data['pc1'][0], data['id_mask1'][0])
    epe = (data['pred_flow'][0] - data['gt_flow'][0,:,:3]).norm(dim=-1) # masking
    # visualize_points3D(data['pc1'][0], epe)
    print(My_metric.get_metric())
    
    path = exp_folder + '/' + get_datetime()
    df = My_metric.get_metric()
    
    os.makedirs(f'{path}', exist_ok=True)
    df.to_csv(path + '/metric.csv')
    
    import pandas as pd
    cfg_df = pd.DataFrame(cfg, index=[0])
    cfg_df.to_csv(path + '/config.csv')
    
    
    
    

  0%|          | 0/552 [00:03<?, ?it/s]


     EPE      AS     AR  Angle    Out  Eval_Time
0  0.178  63.614  74.46  0.313  0.024       1.97


In [ ]:
# metric DO NOT delete
# import glob
# import numpy as np
# import pandas as pd
# exp_folder = f'/mnt/personal/vacekpa2/experiments/' + 'development_metric'
# all_paths = sorted(glob.glob(exp_folder + '/*'))
# 
# for path in all_paths:
#     
#     metric_df = pd.read_csv(path + '/metric.csv', index_col=0)
#     cfg_df = pd.read_csv('tse.csv', index_col=0).iloc[0]    
    
import torch 

from models.RSNF import NeuralPriorNetwork
from loss.flow import GeneralLoss, chamfer_distance_loss
from models.scoopy.get_model import PretrainedSCOOP
from ops.metric import SceneFlowMetric
from data.dataloader import SFDataset4D
from vis.deprecated_vis import imshow, plt
# cfg
plt.close()
device = torch.device('cuda:0')
net = PretrainedSCOOP().to(device)


dataset = SFDataset4D(dataset_type='kitti_t', n_frames=1, only_first=True)
data = dataset[0]
pc1 = data['pc1'][:1].to(device)
pc2 = data['pc2'][:1].to(device)
# net = NeuralPriorNetwork().to(device)

LossModule = GeneralLoss(pc1,pc2,K=32,sm_normals_K=5, smooth_weight=10., forward_weight=1, pc2_smooth=True)
metric = SceneFlowMetric()
# todo cfgs, ablation 
# kitti cfg, ablaiton on weights
net.update(pc1, pc2)
optimizer = torch.optim.Adam(net.parameters(), lr=0.008)

for i in tqdm(range(10)):
    
    pred_flow = net(pc1)
    data['pred_flow'] = pred_flow
    # dist_loss, _, _ = chamfer_distance_loss(pc1 + pred_flow, pc2, both_ways=True)
    loss = LossModule(pc1, pred_flow, pc2)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # print(i, loss)
    # criterion of convergence
metric.update(data) # per one iteration
    
metric.get_metric()
plt.plot(metric.get_metric()['EPE'])
imshow(plt)
# visualize_flow3d(pc1[0], pc2[0], pred_flow[0])

# Instance segmentace
# Rigid instance v case
# 1) ST clustering, smoothnes, dbscan, ID indicator is enough
# 2) Rigid centroids as in rigid loss
# 3) Regularize flow over centroids which should backpropagate directly to flow
# 4) Trajectory consistency keep objects with missing next measurement headed in the same direction


In [87]:


pred_mask = torch.rand((100000, 3), device=device)
centroids = torch.rand((800, 3), device=device)


# find closest centroids to pred_mask
centroids = centroids.unsqueeze(0).repeat(pred_mask.shape[0], 1, 1)
pred_mask = pred_mask.unsqueeze(1).repeat(1, centroids.shape[1], 1)
%timeit dist = (centroids - pred_mask).norm(dim=-1, p=1).argmin(dim=-1)
dist.shape

# db_mask = DBSCAN(eps=0.8, min_samples=30).fit_predict(data['pc1'][0].detach().cpu().numpy())
# visualize_points3D(data['pc1'][0], db_mask)

3.84 ms ± 414 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


torch.Size([100000, 800])

# Freespace

In [9]:
# dataset = SFDataset4D(dataset_type='waymo', n_frames=1, only_first=True)
# data = dataset[4]
pc1 = data['pc1']
pc2 = data['pc2']
flow = data['pred_flow']
nbr_pts = 100
meters = 3
ray_flow = pc2.repeat(nbr_pts, 1, 1)
indices = torch.arange(0, end=nbr_pts, step=1, device=pc2.device).unsqueeze(-1).unsqueeze(-1) / nbr_pts
ray_flow_points = ray_flow * indices 

# inside?
from loss.flow import DT

pc = pc2

# calc_depth = pc[:,:3].norm(dim=1)
# yaw = - torch.arctan2(pc[:, 1], pc[:, 0])
# pitch = torch.arcsin(pc[:, 2] / (calc_depth + 1e-8))
# 
# px = (yaw - yaw.min()) / self.vert_fov * (self.W - 1)
# py = (pitch - pitch.min()) / self.hor_fov * (self.H - 1)        

# DT_layer = DT(pc1 + flow, ray_flow_points.view(1,-1,3))
# dt_loss, dist = DT_layer.torch_bilinear_distance(pc1 + flow)

# visualize_points3D(pc1[0]+flow[0], dist)
# visualize_flow3d(data['pc1'][0], data['pc2'][0], data['pred_flow'][0])
# visualize_multiple_pcls(*[(pc1+flow)[0].detach().cpu().numpy(), pc1[0,:1].detach().cpu().numpy(), ray_flow_points.view(-1,3).detach().cpu().numpy(), pc2[0].detach().cpu().numpy()])
visualize_multiple_pcls(*[(pc1+flow)[0].detach().cpu().numpy(), pc2[0].detach().cpu().numpy()])

# Notes
- rigidity works even for one points
- maybe weight unmatched clusters?
- Waymo 4 sequence case for freespace
    - Only unique first encountered points, measurements are noisy in distances  

# All cluster rigidity loss

In [2]:

# from vis.deprecated_vis import *
# %matplotlib notebook
from IPython.display import HTML
import matplotlib.animation
from tqdm import tqdm
from models.MBNSF.utils import sc_utils
from sklearn.cluster import DBSCAN
from torch_scatter import scatter
pc1 = data['pc1']
pc2 = data['pc2']
cluster_ids = DBSCAN(0.8, min_samples=30).fit_predict(pc1[0].detach().cpu().numpy())
cluster_ids = torch.from_numpy(cluster_ids).float().clone().to(pc1.device)
data['pred_inst'] = cluster_ids.unsqueeze(0).clone()
cluster_ids = data['pred_inst'] + 1 # 0 is noise points, maybe drop all together. In Instance seg, its a background usually
cluster_ids = cluster_ids.long()
# cluster_ids = data['pred_inst']
flow = data['pred_flow'].clone().detach().requires_grad_(True)
optimizer = torch.optim.Adam([flow], lr=0.003)

# visualize_points3D(pc1[0], cluster_ids[0])

In [32]:
def center_rigidity_loss(pc1, flow, cluster_ids):
    '''
    For batch size of 1
    :param pc1: 
    :param flow: 
    :param cluster_ids: 
    :return: 
    '''
    pts_centers = scatter(pc1, cluster_ids, dim=1, reduce='mean')
    flow_centers = scatter(pc1 + flow, cluster_ids, dim=1, reduce='mean')
    
    pt_dist_to_center = (pc1 - pts_centers[0, cluster_ids[0]].unsqueeze(0))#.norm(dim=-1, p=1)
    flow_dist_to_center = ((pc1 + flow) - flow_centers[0, cluster_ids[0]].unsqueeze(0))#.norm(dim=-1, p=1)
    
    center_displacement = pt_dist_to_center - flow_dist_to_center
    
    rigidity_loss = center_displacement.norm(dim=-1).mean()

    return rigidity_loss

In [38]:
# opt_flow = flow.clone().detach().requires_grad_(True)
init_flow = torch.rand(flow.shape, device=pc1.device) + 2
opt_flow = init_flow.clone().requires_grad_(True)
optimizer = torch.optim.Adam([opt_flow], lr=0.01)

# nice!!!
for i in tqdm(range(100)):
    rigid_loss = center_rigidity_loss(pc1, opt_flow, cluster_ids)
    rigid_loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()
    
visualize_multiple_pcls(*[pc1[0].detach().cpu().numpy(), (pc1[0] + opt_flow[0]).detach().cpu().numpy(), (pc1[0] + init_flow[0]).detach().cpu().numpy()])


100%|██████████| 100/100 [00:00<00:00, 102.42it/s]


In [36]:
visualize_points3D(pc1[0], cluster_ids[0])

In [4]:

# from vis.deprecated_vis import *
# %matplotlib notebook
from IPython.display import HTML
import matplotlib.animation
from tqdm import tqdm
from models.MBNSF.utils import sc_utils
from sklearn.cluster import DBSCAN
pc1 = data['pc1']
pc2 = data['pc2']
cluster_ids = DBSCAN(0.8, min_samples=30).fit_predict(pc1[0].detach().cpu().numpy())
cluster_ids = torch.from_numpy(cluster_ids).float().clone().to(pc1.device)
data['pred_inst'] = cluster_ids.unsqueeze(0).clone()
cluster_ids = data['pred_inst']
# cluster_ids = data['pred_inst']
flow = data['pred_flow']

# visualize_points3D(pc1[0], cluster_ids)
object_id = 128
c_pc = pc1[0, cluster_ids[0] == object_id]
c_flow = flow[0, cluster_ids[0] == object_id].clone().detach().requires_grad_(True)

optimizer = torch.optim.Adam([c_flow], lr=0.003)

d_thre = 0.03
src_keypts = c_pc.unsqueeze(0)
tgt_keypts = (c_pc + c_flow).unsqueeze(0)
fig, ax = plt.subplots(1,2)

for i in tqdm(range(45)):
    # iso_loss = sc_utils.spatial_consistency_loss(c_pc.unsqueeze(0), (c_pc + c_flow).unsqueeze(0), d_thre=d_thre)

    # # Spatial Consistency Adjacency Matrix
    src_dist = torch.norm((src_keypts[:, :, None, :] - src_keypts[:, None, :, :]), dim=-1)
    target_dist = torch.norm((tgt_keypts[:, :, None, :] - tgt_keypts[:, None, :, :]), dim=-1)
    cross_dist = torch.abs(src_dist - target_dist)
    adj_mat = torch.clamp(1.0 - cross_dist ** 2 / d_thre ** 2, min=0)
    # divergence is because of lack of |p| = 1?
    # n = adj_mat.shape[1]
    iso_loss = ((adj_mat - 1) ** 2).mean()
    # print(adj_mat.mean())
    loss = - torch.log( iso_loss.mean())
    loss = iso_loss.mean()
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()
    # print(i, loss)
    plt.close()
    plt.axis('equal')
    ax[0].imshow(adj_mat[0][:512,:512].detach().cpu().numpy())
    ax[0].set_title(loss.item())
    ax[1].scatter(c_pc[:,0].detach().cpu().numpy(), c_pc[:,1].detach().cpu().numpy(), c=c_pc[:,2].detach().cpu().numpy(), s=1)
    ax[1].quiver(c_pc[:,0].detach().cpu().numpy(), c_pc[:,1].detach().cpu().numpy(), c_flow[:,0].detach().cpu().numpy(), c_flow[:,1].detach().cpu().numpy(), color='r',
                 scale=1, scale_units='xy')
    path = '/mnt/personal/vacekpa2/vis/current/'
    fig.savefig(f'{path}/{i}.png', dpi=100)
    ax[0].cla()
    ax[1].cla()
    


path = '/mnt/personal/vacekpa2/vis/current/'
# plt.savefig(f'{path}/{i}.png', dpi=200)
# imshow(fig=plt)
# plt.close()
# plt.imshow(adj_mat[0].detach().cpu().numpy())
# imshow(fig=plt)

# if os.path.exists(f'{path}/out.mp4'):
#     os.remove(f'{path}/out.mp4')

# os.system(f"ffmpeg -framerate 15 -pattern_type glob -i '{path}/*.png' -c:v libx264 -pix_fmt yuv420p {path}/out.mp4 > /dev/null")
# HTML(ani.to_jshtml())

# plt.savefig('test.png')

# visualize_flow3d(c_pc, c_pc + c_flow, c_flow)
# visualize_points3D(c_pc, c_pc[:, 2])
# visualize_flow3d(pc1[0], pc2[0], flow[0])

100%|██████████| 45/45 [00:07<00:00,  5.99it/s]


# Per cluster rigidity loss

In [5]:
from ops.transform import xyz_axis_angle_to_matrix, matrix_to_xyz_axis_angle

object_id = 128
c_pc = pc1[0, cluster_ids[0] == object_id]
c_flow = flow[0, cluster_ids[0] == object_id].clone().detach().requires_grad_(True)

c_flow = torch.rand(c_pc.shape, device=c_flow.device) * 0.3
c_flow = c_flow.float().unsqueeze(0).requires_grad_(True)
yaw_rotation = torch.zeros(1, 3, device=c_flow.device, requires_grad=True)
optimizer = torch.optim.Adam([c_flow, yaw_rotation], lr=0.1)

src_keypts = c_pc.unsqueeze(0)
tgt_keypts = (c_pc + c_flow).unsqueeze(0)
# how to deal with Noisy points?


# todo instance probs, allowed rotation of displacement
random_yaw = torch.zeros(1, 3, device=c_flow.device)
random_yaw[0, 2] = 0.4
random_translation = torch.rand(1, 3, device=c_flow.device)

transform = xyz_axis_angle_to_matrix(torch.cat((random_translation, random_yaw), dim=1)).to(device)

target_pc = (c_pc[:,:3] @ transform[0, :3,:3] + transform[0,:3,3]).unsqueeze(0) # transform okay I guess

# todo continue here
yaw = yaw_rotation[0, 2]
inst_probs = torch.ones(1, c_pc.shape[0], dtype=torch.float, device=c_flow.device)

# print(target_pc.shape)
# visualize_multiple_pcls(*[c_pc.detach().cpu().numpy(), target_pc[0].detach().cpu().numpy()])

plt.close()
fig, ax = plt.subplots(1,2) 

for i in tqdm(range(100)):
    
    center = (src_keypts.mean(dim=1)).unsqueeze(1)
    # target_pt = (src_keypts.mean(dim=1) + 1).unsqueeze(1)
    flow_center = ((src_keypts + c_flow).mean(dim=1)).unsqueeze(1)
    
    rot_mat = (xyz_axis_angle_to_matrix(torch.cat((torch.zeros(1, 3, device=c_flow.device), yaw_rotation), dim=1)).to(device))
    
     
    
    rotated_flows = ((c_flow - c_flow.mean(dim=1)) @ rot_mat[:,:3,:3]) + c_flow.mean(dim=1)
    
    
    
    # dist_to_center = (center[..., :2] - src_keypts[..., :2]).norm(dim=-1, p=1)
    # flow_dist_to_center = (flow_center[..., :2] - (src_keypts[..., :2] + c_flow[..., :2])).norm(dim=-1, p=1)
    # exact dimension from center
    residual_from_center = (center[..., :2] - src_keypts[..., :2]) - (flow_center[..., :2] - (src_keypts[..., :2] + c_flow[..., :2]))
    # rigidity_loss = (dist_to_center - flow_dist_to_center).abs()
    rigidity_loss = residual_from_center.abs().mean(dim=-1)
    
    # version with ||d||
    # rigidity_loss = ((center[..., :2] - src_keypts[..., :2]).norm(dim=-1) - (flow_center[..., :2] - (src_keypts[..., :2] + c_flow[..., :2])).norm(dim=-1)).abs()
    # maybe enought with NN
    
    # flow_dist_to_target = (target_pc[..., :2] - (src_keypts[..., :2] + c_flow[..., :2])).norm(dim=-1, p=1)
    flow_dist_to_target, nn_ind, _ = knn_points((src_keypts[..., :2] + c_flow[..., :2]), target_pc[..., :2])
    

    rot_dist, _, _ = knn_points(rotated_flows[..., :2], target_pc[..., :2])
    # push all residual to the mean (points should move isometrically and no point should diverge from the mean)
    # loss = (residual_dist - mean_residual).mean() #+ flow_dist_to_center.mean()
    # loss = rigidity_loss.mean()
    loss = flow_dist_to_target.mean() + 3 * rigidity_loss.mean() + rot_dist.mean() + yaw_rotation[:,:2].abs().mean()
    # residual_dist.shape, flow_dist_to_center.shape, dist_to_center.shape, mean_residual
    loss.backward()
    
    print(yaw_rotation)
    
    optimizer.step()
    optimizer.zero_grad()
    # # print(i, loss)
    plt.close()
    plt.axis('equal')
    
    ax[0].plot(rigidity_loss[0].detach().cpu().numpy(), '.')
    ax[0].set_ylim(0, 0.4)
    ax[1].axis('equal')
    # ax[0].imshow(adj_mat[0][:512,:512].detach().cpu().numpy())
    
    ax[1].set_title(f"Iter: {i:03d} Loss: {loss.item():.4f}")
    # ax[1].set_xlim(21, 24)
    # ax[1].set_ylim(8, 10)
    ax[1].scatter(c_pc[:,0].detach().cpu().numpy(), c_pc[:,1].detach().cpu().numpy(), c=rigidity_loss[0].detach().cpu().numpy(), s=5)
    ax[1].scatter(center.view(-1,3)[:,0].detach().cpu().numpy(), center.view(-1,3)[:,1].detach().cpu().numpy(), c='k', marker='*', s=30)
    ax[1].scatter(flow_center.view(-1,3)[:,0].detach().cpu().numpy(), flow_center.view(-1,3)[:,1].detach().cpu().numpy(), c='g', marker='*', s=30)
    ax[1].scatter(target_pc.view(-1,3)[:,0].detach().cpu().numpy(), target_pc.view(-1,3)[:,1].detach().cpu().numpy(), c='r', marker='.', s=5)
    ax[1].scatter((c_pc + rotated_flows.view(-1,3))[:,0].detach().cpu().numpy(), (c_pc + rotated_flows.view(-1,3))[:,1].detach().cpu().numpy(), c='k', marker='.', s=5, alpha=0.2)
    ax[1].scatter((c_pc[:, 0] +  c_flow[0,:,0]).detach().cpu().numpy(), (c_pc[:, 1] +  c_flow[0,:,1]).detach().cpu().numpy(), c='g', marker='.', s=5)
    
    # faking quiver is wrong animation
    # ax[1].quiver(c_pc[:,0].detach().cpu().numpy(), c_pc[:,1].detach().cpu().numpy(), c_flow[0,:,0].detach().cpu().numpy(), c_flow[0,:,1].detach().cpu().numpy(), color='g',
    #              scale=1, scale_units='xy')
    path = '/mnt/personal/vacekpa2/vis/current/'
    fig.savefig(f'{path}/{i}.png', dpi=100)
    ax[0].cla()
    ax[1].cla()
    

  1%|          | 1/100 [00:00<00:13,  7.49it/s]

tensor([[0., 0., 0.]], device='cuda:0', requires_grad=True)
tensor([[ 0.1000, -0.1000,  0.1000]], device='cuda:0', requires_grad=True)


  3%|▎         | 3/100 [00:00<00:12,  7.62it/s]

tensor([[ 0.0256, -0.0256,  0.1841]], device='cuda:0', requires_grad=True)
tensor([[-0.0602,  0.0602,  0.2061]], device='cuda:0', requires_grad=True)


  5%|▌         | 5/100 [00:00<00:12,  7.71it/s]

...

tensor([[0.0073, 0.0291, 1.8083]], device='cuda:0', requires_grad=True)
tensor([[0.0159, 0.0177, 1.8089]], device='cuda:0', requires_grad=True)


 98%|█████████▊| 98/100 [00:13<00:00,  7.70it/s]

tensor([[ 0.0135, -0.0025,  1.8098]], device='cuda:0', requires_grad=True)
tensor([[ 1.3336e-03, -1.0605e-02,  1.8108e+00]], device='cuda:0',
       requires_grad=True)


100%|██████████| 100/100 [00:13<00:00,  7.39it/s]

tensor([[-0.0196, -0.0078,  1.8118]], device='cuda:0', requires_grad=True)


# Case study

In [2]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from pytorch3d.transforms import axis_angle_to_matrix
from sklearn.cluster import DBSCAN
from data.PATHS import DATA_PATH
from loss.flow import *

from vis.deprecated_vis import *
from models.simple_models import InstanceSegModule, FlowSegPrior, Weights_model, JointFlowInst
import matplotlib.pyplot as plt

from loss.flow import chamfer_distance_loss, GeneralLoss, FastNN
from loss.instance import DynamicLoss, InstanceSmoothnessLoss
from loss.utils import find_robust_weighted_rigid_alignment
from models.RSNF import NeuralPriorNetwork,RigidMovementPriorNetwork
from loss.flow import DT

from torch_scatter import scatter
from ops.metric import SceneFlowMetric
from data.dataloader import NSF_dataset

class JointModel(torch.nn.Module):

    def __init__(self, pc1, instances=10, init_cluster=None):
        super().__init__()
        self.instances = instances
        
        if init_cluster:
            # print('clustering')
            # oversegmentation
            clusters = DBSCAN(0.3, min_samples=5).fit_predict(pc1[0].detach().cpu().numpy()) + 1
            
            instances = clusters.max()
            # split to mask
            mask = torch.zeros((1, pc1.shape[1], instances), device=device)
            for i in range(instances):  # can be speed up
                mask[0, clusters == i, i] = 1
            self.mask = torch.nn.Parameter(mask, requires_grad=True)
            # visualize_points3D(pc1[0], mask.argmax(dim=2)[0])
        else:
            self.mask = torch.nn.Parameter(torch.randn(1, pc1.shape[1], instances, requires_grad=True))
            
        self.flow_net = RigidMovementPriorNetwork()

        # todo add boxes here?
    def forward(self, pc1, pc2=None):

        output = self.flow_net(pc1)
        mask = self.mask.softmax(dim=2)
        
        # Assign rigid parameters
        t = output[:,:, :3]
        yaw = output[:,:, 3:4]
        
        # construct rotation
        full_rotvec = torch.cat((torch.zeros((1, yaw.shape[1], 2), device=device), yaw), dim=-1)
        rot_mat = axis_angle_to_matrix(full_rotvec.squeeze(0)) # this is lot, but still okay
        
        # construct centroids
        ind = torch.argmax(mask, dim=2).squeeze(0)
        v_max = scatter(pc1[0], ind, dim=0, reduce='max')  
        v_min = scatter(pc1[0], ind, dim=0, reduce='min') 
        x_c = (v_max + v_min) / 2
        
        # shift points
        point_x_c = x_c[ind]
        
        
        deformed_pc = (rot_mat @ (pc1 - point_x_c).permute(1, 2, 0)).permute(2, 0, 1) + point_x_c + t
        
        pred_flow = deformed_pc - pc1
        
        return pred_flow, mask, t, yaw
        # return pred_flow, mask

if __name__ == '__main__':
    if torch.cuda.is_available():
        device = torch.device(0)
    else:
        device = torch.device('cpu')
    
    metric = SceneFlowMetric()
    dataset_type = 'ft3d'
    # dataset = NSF_dataset(dataset_type=dataset_type)
    dataset = NSF_dataset(dataset_type=dataset_type)
    
    K = 32
    max_radius = 1
    grid_factor = 10
    
    # just quick iteration over dataset
    for f, data in enumerate(tqdm(dataset)):
        # *_, data = dataset
        # if f != 26:
        #     continue
        
        pc1 = data['pc1'].to(device)
        pc2 = data['pc2'].to(device)
        gt_flow = data['gt_flow'].to(device)
    
        
        st = time.time()
        model = JointModel(pc1, instances=30, init_cluster=True).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        # losses
        # sm_normals_K = 4 for sota
        LossModule = GeneralLoss(pc1=pc1, pc2=pc2, dist_mode='DT', K=K, max_radius=max_radius, smooth_weight=1, forward_weight=0, sm_normals_K=0, pc2_smooth=True)
        
        
        for flow_e in range(150):
    
            pc1 = pc1.contiguous()
            pred_flow, mask, t, yaw = model(pc1)    # Model outputs directly mask and predicted flow
            
            # for mask prob
            # mask_probs = mask.max(dim=2)[0]
            # forw_dist, forward_nn, _ = knn_points(pc1 + pred_flow, pc2, lengths1=None, lengths2=None, K=1, norm=1)
            # back_dist, backward_nn, _ = knn_points(pc2, pc1 + pred_flow, lengths1=None, lengths2=None, K=1, norm=1)
            # dist_loss = ((forw_dist).mean() + (back_dist).mean()) / 2
            
            # dist_loss = ((forw_dist * mask_probs).mean() + (back_dist).mean()) / 2
            
            data['pred_flow'] = pred_flow
            
            loss = LossModule(pc1, pred_flow, pc2)
            inst_smooth, _ = LossModule.smoothness_loss(mask, LossModule.NN_pc1)    # added instances
            trans_smooth, _ = LossModule.smoothness_loss(t, LossModule.NN_pc1)    # added instances
            yaw_smooth, _ = LossModule.smoothness_loss(yaw, LossModule.NN_pc1)    # added instances
            
            # regularizations
            # loss += inst_smooth.mean() + trans_smooth.mean() + yaw_smooth.mean() + (yaw ** 2).mean() #+ (pred_flow.norm(dim=-1)).mean() * 10
            
            
            loss.mean().backward()
    
            optimizer.step()
            optimizer.zero_grad()
    
            # print(f"Iter: {flow_e:03d} \t Loss: {loss.mean().item():.4f} \t Flow: {loss.mean().item():.4f} "
            #       f"\t Smoothness: {inst_smooth.mean().item():.4f} "
                  # f"Cycle: {cycle_smooth_flow.mean().item():.4f} \t Dynamic: {dynamic_loss.mean().item():.4f} \t"
                  # f"RMSD: {rmsd.mean().item():.4f} \t Kabsch_w: {kabsch_w.mean().item():.4f}"
                  # )
            
        data['eval_time'] = time.time() - st
        data['pc1'] = pc1
        data['pc2'] = pc2
        data['gt_flow'] = gt_flow
        metric.update(data)
        
        break
        
    print(metric.get_metric())
    print(metric.get_metric().mean())
    instance_classes = torch.argmax(mask, dim=2)[0].squeeze(0).detach().cpu().numpy()
    # visualize_points3D(pc1[0].detach().cpu().numpy(), instance_classes)
    # visualize_flow3d(pc1[0].detach().cpu().numpy(), pc2[0].detach().cpu().numpy(), pred_flow[0].detach().cpu().numpy())
    # visualize_points3D(pc1[0].detach().cpu().numpy(), instance_classes == 0)


  0%|          | 0/2007 [00:00<?, ?it/s]


KeyError: 'flow'

In [31]:
instance_classes = torch.argmax(mask, dim=2)[0].squeeze(0).detach().cpu().numpy()

# visualize_points3D(pc1[0].detach().cpu().numpy(), instance_classes)
# visualize_flow3d(pc1[0].detach().cpu().numpy(), pc2[0].detach().cpu().numpy(), pred_flow[0].detach().cpu().numpy())
# visualize_flow3d(pc1[0].detach().cpu().numpy(), pc2[0].detach().cpu().numpy(), gt_flow[0].detach().cpu().numpy())
# df_mask = (metric.get_metric()['EPE'] < 0.2)
# metric.get_metric()[df_mask].mean()
metric.get_metric().mean()

EPE           0.026
AS           97.608
AR           98.571
Angle         0.109
Out           0.104
Eval_Time     4.801
dtype: float64

In [ ]:
# centroids
from torch_scatter import scatter

a = c_pc[0, mos]
# ids from dbscan
ind = torch.tensor(ids, device=device) + 1  # what about -1?

# print(a.shape, ind.shape)
# print(ind.max())
# centroid = scatter(a, ind, dim=0, reduce='mean')
v_max = scatter(a, ind, dim=0, reduce='max')  
v_min = scatter(a, ind, dim=0, reduce='min') 

flow_vectors = scatter(forth_flow[0, mos], ind, dim=0, reduce='mean')

flow_magnitudes = flow_vectors.norm(dim=1, p=1)
yaw_from_flow = torch.arctan2(flow_vectors[:,1], flow_vectors[:,0]) # in radians

yaw_from_flow[ind]
box_corners = torch.stack([v_max, v_min], dim=1)    # for vis?
centroids = (v_max + v_min) / 2

# lwh - rotate points around centroids with flow(~yaw) angle and find max and min in dimensions
# can be extended to 3D,for simplicity only 2D now
rot_mat = torch.stack([torch.cos(yaw_from_flow), -torch.sin(yaw_from_flow), torch.sin(yaw_from_flow), torch.cos(yaw_from_flow)], dim=1).view(-1, 2, 2)

# rotate points
broadcasted_rot_mat = rot_mat[ind]
shifted_points = a - centroids[ind]

res = broadcasted_rot_mat @ shifted_points[:,:2].unsqueeze(-1)

res = torch.cat((res, a[:, 2:3].unsqueeze(-1) - centroids[ind, 2:3].unsqueeze(-1)), dim=1).squeeze(-1) + centroids[ind]

# ---------------------------------------------------
# do not forget about noise points, that are now ind 0!!!
# ---------------------------------------------------

# print(broadcasted_rot_mat.shape, shifted_points.shape, res.shape)
visualize_multiple_pcls(*[c_pc[0, mos].detach().cpu().numpy(), centroids.detach().cpu().numpy(), res.detach().cpu().numpy()])

# rotated points
shifted_limits_max = scatter(res, ind, dim=0, reduce='max')
shifted_limits_min = scatter(res, ind, dim=0, reduce='min')

lwh = shifted_limits_max - shifted_limits_min

boxes = torch.cat((centroids, lwh, yaw_from_flow.unsqueeze(-1)), dim=1)

from data.box_utils import get_bbox_points
box_points = get_bbox_points(boxes[1:].detach().cpu())  # drop noise points
visualize_multiple_pcls(*[c_pc[0, mos].detach().cpu().numpy(), centroids.detach().cpu().numpy(), box_points])
# visualize_points3D(box_points, box_points[:,3])





# Next steps
- Iterative spatio-temporal clustering with warped flow

- box after rotation (x=16.25, y=-8.8, z=0.3)
- gradiently enlarge boxes 
- Freespace to divide boxes or make them smaller
- Linear motion
- NP experiments
- How to merge, how to coverage?
- Coverage, calculate DT/NN between boxes and points, increase coverage to nearby points